# Replicando dbo.SP_KPY_DABOGADOJUD_STG
```sql
--1-----------------[dbo].[SP_KPY_DABOGADOJUD_STG]--------------------------

CREATE PROCEDURE [dbo].[SP_KPY_DABOGADOJUD_STG]
AS
SET NOCOUNT ON
BEGIN

-- INSERTA LOG CON EL INICIO EJECUCION
INSERT INTO CONTROL_EJECUCION_MALLA_DS
    (NOMBRE_JOB, NOMBRE_WORKFLOW, FECHAHORA_INICIO, FECHAHORA_FIN, FECHA_PROCESO)
VALUES
    ('JOB_MALLA_CREDITOS', OBJECT_NAME(@@PROCID), GETDATE(), NULL, GETDATE())

-- 34_STG_DABOGADOJUD
TRUNCATE TABLE STG_DABOGADOJUD

INSERT INTO STG_DABOGADOJUD
SELECT
    cCodPerJud AS CODABOGADOJUD,
    CASE 
        WHEN cNomPerJud = '' OR cNomPerJud IS NULL THEN '.'
        ELSE REPLACE(TRIM(cNomPerJud), ',', ' ')
    END NOMABOGADOJUD,
    GETDATE() AS FECULTACTUALIZACION
FROM SOFMACHAYO_BI_SNAP.DBO.KJUDPPerJudici
WHERE (cCodTipPer = 'ABO')

UPDATE CONTROL_EJECUCION_MALLA_DS
SET FECHAHORA_FIN = GETDATE()
WHERE NOMBRE_JOB = 'JOB_MALLA_CREDITOS'
    AND NOMBRE_WORKFLOW = OBJECT_NAME(@@PROCID)
    AND FECHA_PROCESO = CAST(GETDATE() AS DATE)

END
--------------------------------------------------------------------------
```

## Paso 1. Preparando entorno

In [1]:
# Obtención de Spark Session configurado
import os
os.environ['PYSPARK_PYTHON'] = 'python3.11.8'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.11.8'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
from utils import get_spark, read_query
spark = get_spark()

## Paso 2. Estableciendo accesos

In [2]:
# Parametros de conexión al SOURCE (SQL SERVER)
USER="SA"
PASSWORD="MUBL77XSPRA9gary"
URL="jdbc:sqlserver://mssql-service.data-services.svc.cluster.local:1433;databaseName=SOFMACHAYO_BI_SNAP"

## Paso 3. Estableciendo parámetros

In [3]:
BUCKET = os.getenv("BUCKET", default="lhchdev")
CAPA = "brz"
BASEDATOS = "brz_db"
FUENTE = "fuente"
TABLA = "KJUDPPerJudici"

## Paso 4. Creando tabla en capa bronze

In [4]:
spark.sql("create database if not exists brz_db;")

DataFrame[]

In [6]:
spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {BASEDATOS}.{FUENTE}_{TABLA} (
    cCodTipPer STRING COMMENT 'Código de tipo de persona',
    cCodPerJud STRING COMMENT 'Código del perito judicial',
    cNomPerJud STRING COMMENT 'Nombre completo del perito judicial'
)
USING DELTA
COMMENT 'Tabla de peritos judiciales'
LOCATION 's3a://{BUCKET}/{CAPA}/{FUENTE}/{TABLA}'
;""")

DataFrame[]

## Paso 5 (Opcional). Describe la metadata de tu tabla

In [7]:
spark.sql("DESCRIBE EXTENDED brz_db.fuente_kjudpperjudici;").show(100,False)

+----------------------------+---------------------------------------------------+-----------------------------------+
|col_name                    |data_type                                          |comment                            |
+----------------------------+---------------------------------------------------+-----------------------------------+
|cCodTipPer                  |string                                             |Código de tipo de persona          |
|cCodPerJud                  |string                                             |Código del perito judicial         |
|cNomPerJud                  |string                                             |Nombre completo del perito judicial|
|                            |                                                   |                                   |
|# Detailed Table Information|                                                   |                                   |
|Name                        |spark_catalog.brz_

## Paso 6. Poblamiento de la tabla
- Tipo Overwrite

In [8]:
df = read_query(query="""(
    SELECT * FROM SOFMACHAYO_BI_SNAP.DBO.KJUDPPerJudici
)dym""",spark=spark,url=URL,user=USER,password=PASSWORD)

In [9]:
df.write.format("delta").insertInto("brz_db.fuente_kjudpperjudici", overwrite=True)